<a href="https://colab.research.google.com/github/sambhavigit/LKMAIGCP/blob/main/Batch4/text_to_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vertex AI Model Garden - Text To Video

<table align="left">
  <td>
Colab Notebook

## Overview

This notebook demonstrates deploying the pre-trained [Text To Video](https://huggingface.co/docs/diffusers/main/en/api/pipelines/text_to_video) model on Vertex AI for online prediction.

### Objective

- Upload the model to [Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction).
- Deploy the model on [Endpoint](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for text-to-video.


In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

In [ ]:
# Cloud project id.
PROJECT_ID = "acn-lkmaigcp"  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = "us-east1"  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

Initialize Vertex AI API:

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project="acn-lkmaigcp", location="us-east1"  , staging_bucket=GCS_BUCKET)

### Define constants

In [ ]:
# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-diffusers-serve"

In [ ]:
from google.cloud import aiplatform


def deploy_model(model_id, task):
    model_name = "text-to-video"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
    }
    # If the model_id is a GCS path, use artifact_uri to pass it to serving docker.
    artifact_uri = model_id if model_id.startswith("gs://") else None
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/diffusers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
        artifact_uri=artifact_uri,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type="n1-standard-8",
        accelerator_type="NVIDIA_TESLA_T4",
        accelerator_count=1,
        deploy_request_timeout=1800,
    )
    return model, endpoint

In [ ]:
##Creating Endpoint and Model Registration

model, endpoint = deploy_model(
    model_id="damo-vilab/text-to-video-ms-1.7b", task="text-to-video"
)

In [ ]:
##Prompt

instances = [
    {"prompt": "Spiderman and batman is surfing"},
]
response = endpoint.predict(instances=instances)

In [ ]:
from IPython.display import HTML

html = ""
for video in response.predictions:
    html += "<video controls>"
    html += f'<source src="data:video/mp4;base64,{video}" type="video/mp4">'
    html += "</video>"
HTML(html)

### Clean up resources if not using this model further for any video creation

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()